## Approche Signal pour Parkinson
---

In [ ]:
import os
import librosa
import numpy as np

# Chemins des dossiers
path_to_hc = 'HC_AH/'
path_to_pd = 'PD_AH/'

# Fonction pour charger les fichiers audio
def charger_audio_files(fichier):
    signals = []
    for file in os.listdir(fichier):
        if file.endswith('.wav'):
            audio_path = os.path.join(fichier, file)
            y, sr = librosa.load(audio_path, sr=None)  
            signals.append(y)
    return signals

# Charger les fichiers audio en utilisant la fonction
hc_signals = charger_audio_files(path_to_hc)
pd_signals = charger_audio_files(path_to_pd)

# Combiner les données
X = hc_signals + pd_signals  # Liste des signaux audio
y = np.array([0] * len(hc_signals) + [1] * len(pd_signals))  # Labels (0 sain, 1 parkinsonien)

In [ ]:
print('Nombre de signaux audio:', len(X))
print('Nombre de labels:', len(y))

print('Exemple de signal audio:', X[0])
print('Exemple de label:', y[0])

In [ ]:
# Une longueur commune pour tous les signaux
fixed_length = 8000 * 3  # 3 secondes à 8000Hz = 24000 échantillons

# Fonction pour redimensionner les signaux
def fix_signal_length(signal, fixed_length):
    if len(signal) > fixed_length:
        signal = signal[:fixed_length]  # Tronquer selon longueur fixée
    else:
        signal = np.pad(signal, (0, max(0, fixed_length - len(signal))))  # Remplir avec des zéros si la longueur est inférieure
    return signal

# Appliquer la fonction à tous les signaux
X_fixed = [fix_signal_length(signal, fixed_length) for signal in X]
X_fixed = np.array(X_fixed)  # Convertir en tableau NumPy

In [ ]:
from sklearn.model_selection import train_test_split

# Diviser les données
X_train, X_val, y_train, y_val = train_test_split(X_fixed, y, test_size=0.3, random_state=42, stratify=y)

# Vérifier les formes
print("Shape de X_train :", X_train.shape)
print("Shape de X_val :", X_val.shape)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Fonction pour extraire les MFCC
def extract_mfcc(signals, sr=8000, n_mfcc=13):
    mfccs = []
    for signal in signals:
        mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
        mfccs.append(np.mean(mfcc.T, axis=0))  # Moyenne des MFCC pour chaque signal
    return np.array(mfccs)

# Extraire les MFCC pour les ensembles d'entraînement et de validation
X_train_mfcc = extract_mfcc(X_train)
X_val_mfcc = extract_mfcc(X_val)

# Entraîner un SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_mfcc, y_train)

# Faire des prédictions
y_pred_svm = svm_model.predict(X_val_mfcc)

# Évaluer le modèle
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print("Accuracy du SVM :", accuracy_svm)

---

Modèle CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Redimensionner les données pour le CNN
X_train_cnn = X_train[..., np.newaxis]  # Ajouter une dimension pour les canaux
X_val_cnn = X_val[..., np.newaxis]

# Définir l'architecture du CNN
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(fixed_length, 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    X_train_cnn, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_cnn, y_val)
)

# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(X_val_cnn, y_val)
print("Accuracy du CNN :", test_accuracy)

---

Modèle CNN appliqué sur les spectrogrammes

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Convertir les signaux en spectrogrammes
def audio_to_spectrogram(signal, sr=8000, n_mels=128):
    S = librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=n_mels)
    S_db = librosa.power_to_db(S, ref=np.max)
    return S_db

# Appliquer à tous les signaux
X_train_spec = np.array([audio_to_spectrogram(signal) for signal in X_train])
X_val_spec = np.array([audio_to_spectrogram(signal) for signal in X_val])

# Redimensionner pour le CNN
X_train_spec = X_train_spec[..., np.newaxis]
X_val_spec = X_val_spec[..., np.newaxis]

# Adapter le CNN pour les spectrogrammes
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train_spec.shape[1:]),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compiler et entraîner le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_spec, y_train, epochs=50, batch_size=32, validation_data=(X_val_spec, y_val))

# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(X_val_spec, y_val)
print("Accuracy du CNN avec spectrogrammes :", test_accuracy)

# Afficher le plot de l'accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

---

Modèle YAMNET

In [ ]:
!pip install tensorflow_hub --user

In [ ]:
import tensorflow_hub as hub

# Charger YAMNet
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Extraire les embeddings
def extract_embeddings(signals):
    embeddings = []
    for signal in signals:
        _, embedding, _ = model(signal)
        embeddings.append(np.mean(embedding, axis=0))  # Moyenne sur les frames
    return np.array(embeddings)

# Appliquer à vos données
X_train_emb = extract_embeddings(X_train)
X_val_emb = extract_embeddings(X_val)

# Entraîner un SVM sur les embeddings
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_emb, y_train)
y_pred_svm = svm_model.predict(X_val_emb)
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print("Accuracy du SVM avec YAMNet :", accuracy_svm)